This notebook is written based on [this reference implementation](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb).

Other refs for model:
* https://stackoverflow.com/questions/65205582/how-can-i-add-a-bi-lstm-layer-on-top-of-bert-model
* https://discuss.pytorch.org/t/how-to-connect-hook-two-or-even-more-models-together/21033
* https://pytorch.org/tutorials/beginner/transformer_tutorial.html
* https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Other refs for torchtext:
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-i-5da6f1c89d84
* https://towardsdatascience.com/use-torchtext-to-load-nlp-datasets-part-ii-f146c8b9a496
* http://anie.me/On-Torchtext/

# Imports and setup

In [1]:
import pandas as pd
import numpy as np
import os
import random
# random.seed(1)
import re

# Data processing.
import constants # constants.py
import dataset # dataset.py
import torch

# Model.
import models # models.py
import torch.nn as nn
from transformers import DistilBertModel

# Training.
import training # training.py
import utils # utils.py

# If you make a code change that doesn't get picked up by
# Jupyter notebook, try reloading like below:
# import imp
# imp.reload(training)

# Read the data

In [3]:
# data_df = dataset.get_multiple_datasets([1,2,3], 'Creativity_Combined', shuffle=True)

In [4]:
'''This cell is commented out because the csvs should already exist in the directory.
If you are running the notebook for the first time, run them to generate the csvs.'''
# split into train, test sets. (Train set will be further split into 
# train+validation sets, via k-fold CV.)
# train_df = data_df[:1000]
# test_df = data_df[1000:] # roughly 203 test examples set aside

# write them to CSV files
# train_df.to_csv('ktrain.csv', index=False, header=False)
# test_df.to_csv('ktest.csv', index=False, header=False)

## Preprocessing and transform into torchtext Dataset format.

From what I understand, some preprocessing is done when data.Field() is applied.

In [2]:
train_dataset, test_dataset = dataset.get_train_test_datasets()

In [3]:
# Transform train_dataset into an np array representation.
# This will be used for generating the K folds.
train_exs_arr = np.array(train_dataset.examples)

# Training pipeline begins here


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

param_grid = {
    'dropout': [0.2],
    'batch_size': [8],
    'max_epochs': [10],
    'lr': [5e-05],
    'hidden_dim': [128,256,512,768],
    'num_layers': [1,2,3],
    'bidirectional': [False,True],
}

results, best_model = training.perform_hyperparameter_search(param_grid, train_exs_arr, rnn=True, save_weights=True)
print(best_model)

eid 0, params {'batch_size': 8, 'bidirectional': False, 'dropout': 0.2, 'hidden_dim': 128, 'lr': 5e-05, 'max_epochs': 10, 'num_layers': 1}


/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


training on fold 0


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


updating saved weights of best model
Epoch: 00 | Epoch Time: 0m 52s
	 Train Loss: 25.232 | Train Corr: 0.03
	 Val. Loss: 5.094 |  Val. Corr: 0.41
updating saved weights of best model
Epoch: 01 | Epoch Time: 0m 52s
	 Train Loss: 5.509 | Train Corr: 0.04
	 Val. Loss: 4.337 |  Val. Corr: 0.44
updating saved weights of best model
Epoch: 02 | Epoch Time: 0m 52s
	 Train Loss: 4.452 | Train Corr: 0.36
	 Val. Loss: 4.108 |  Val. Corr: 0.42
Epoch: 03 | Epoch Time: 0m 52s
	 Train Loss: 3.905 | Train Corr: 0.47
	 Val. Loss: 4.680 |  Val. Corr: 0.45
updating saved weights of best model
Epoch: 04 | Epoch Time: 0m 51s
	 Train Loss: 3.407 | Train Corr: 0.57
	 Val. Loss: 3.048 |  Val. Corr: 0.60
updating saved weights of best model
Epoch: 05 | Epoch Time: 0m 52s
	 Train Loss: 2.683 | Train Corr: 0.68
	 Val. Loss: 2.912 |  Val. Corr: 0.63
updating saved weights of best model
Epoch: 06 | Epoch Time: 0m 52s
	 Train Loss: 2.020 | Train Corr: 0.77
	 Val. Loss: 2.753 |  Val. Corr: 0.62
Epoch: 07 | Epoch Tim

...

Epoch: 00 | Epoch Time: 2m 8s
	 Train Loss: 16.953 | Train Corr: 0.05
	 Val. Loss: 4.085 |  Val. Corr: 0.20
Epoch: 01 | Epoch Time: 2m 10s
	 Train Loss: 4.719 | Train Corr: 0.22
	 Val. Loss: 4.356 |  Val. Corr: 0.40
Epoch: 02 | Epoch Time: 2m 10s
	 Train Loss: 3.683 | Train Corr: 0.48
	 Val. Loss: 5.084 |  Val. Corr: 0.44
Epoch: 03 | Epoch Time: 2m 10s
	 Train Loss: 3.278 | Train Corr: 0.56
	 Val. Loss: 4.013 |  Val. Corr: 0.50
training on fold 3


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 2m 6s
	 Train Loss: 16.490 | Train Corr: 0.08
	 Val. Loss: 5.250 |  Val. Corr: 0.25
Epoch: 01 | Epoch Time: 2m 5s
	 Train Loss: 3.378 | Train Corr: 0.51
	 Val. Loss: 4.898 |  Val. Corr: 0.28
training on fold 4


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch: 00 | Epoch Time: 2m 2s
	 Train Loss: 16.431 | Train Corr: 0.07
	 Val. Loss: 5.157 |  Val. Corr: 0.44
Epoch: 01 | Epoch Time: 2m 3s
	 Train Loss: 3.308 | Train Corr: 0.53
	 Val. Loss: 4.068 |  Val. Corr: 0.49
Epoch: 02 | Epoch Time: 2m 1s
	 Train Loss: 2.607 | Train Corr: 0.66
	 Val. Loss: 4.563 |  Val. Corr: 0.59
Epoch: 03 | Epoch Time: 2m 1s
	 Train Loss: 1.976 | Train Corr: 0.75
	 Val. Loss: 5.861 |  Val. Corr: 0.55
Epoch: 04 | Epoch Time: 2m 1s
	 Train Loss: 1.308 | Train Corr: 0.85
	 Val. Loss: 3.163 |  Val. Corr: 0.63
updating saved weights of best model
Epoch: 05 | Epoch Time: 2m 3s
	 Train Loss: 0.973 | Train Corr: 0.89
	 Val. Loss: 2.697 |  Val. Corr: 0.70
('batch_size_8; bidirectional_True; dropout_0.2; hidden_dim_768; lr_5e-05; max_epochs_10; num_layers_1', 0.6996348896257089)


# Test the trained model on held-out dataset.

In [6]:
# Get a test iterator
# use batch size from best params!!
test_iterator = training.get_iterator(test_dataset, 8, device)

In [7]:
# load the best model saved
bert = DistilBertModel.from_pretrained(constants.WEIGHTS_NAME)
# use the params from the best model!!! 
model = models.BERTRNN(bert, constants.OUTPUT_DIM, 768, 1, True, 0.2)
model.load_state_dict(torch.load("21_best_valid_loss.pt"))
model.to(device)
model.eval()
# If you change the criterion, make sure it matches with the training criterion in training.py
criterion = nn.MSELoss(size_average=False)
criterion = criterion.to(device)
test_loss, test_corr = training.evaluate(model, test_iterator, criterion, debug=True)
print(test_loss)
print(test_corr)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


predictions: tensor([4.3092, 2.5972, 4.9821, 3.7691, 4.0425, 4.0092, 4.1976, 3.6279],
       device='cuda:0')
true labels: tensor([4.8000, 2.7500, 5.3250, 3.9250, 4.7250, 5.2000, 4.7500, 4.7250],
       device='cuda:0')
predictions: tensor([4.4185, 4.0181, 3.9511, 4.4189, 4.6874, 2.4615, 4.5022, 3.6038],
       device='cuda:0')
true labels: tensor([4.4500, 4.5000, 4.2750, 4.3750, 4.8750, 3.3250, 4.8000, 3.7250],
       device='cuda:0')
predictions: tensor([4.6404, 3.5096, 3.9365, 3.2364, 3.2019, 4.2997, 3.9174, 2.8325],
       device='cuda:0')
true labels: tensor([5.0000, 3.3500, 4.0250, 4.1000, 4.9000, 4.3000, 3.1750, 2.6750],
       device='cuda:0')
predictions: tensor([3.5482, 3.5646, 4.7370, 2.4181, 4.3563, 3.4085, 3.8179, 3.4318],
       device='cuda:0')
true labels: tensor([4.8250, 4.1500, 5.2750, 2.6500, 5.0000, 4.7000, 4.3000, 3.7750],
       device='cuda:0')
predictions: tensor([3.8663, 3.7502, 3.8978, 3.4236, 4.2574, 4.9229, 4.0129, 4.6737],
       device='cuda:0')
true label

# Misc other stuff

Link to the trainer class: https://huggingface.co/transformers/main_classes/trainer.html



Default training arguments: https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments

Batch size per device: 8

Epoch: 3



This should be the model I used to generate my initial results: https://huggingface.co/transformers/model_doc/distilbert.html#distilbertforsequenceclassification
"DistilBert Model transformer with a sequence classification/regression head on top (a linear layer on top of the pooled output) e.g. for GLUE tasks."